# Cook panel under plane stress

In this example we investigate the well-known benchmark of a tapered panel under plane stress conditions known under the name of Cook.  The problem has been solved many times with a variety of finite element models  and hence the solution is well-known.

The problem is solved in a script.  We begin  by `using` the top-level module `FinEtools` and also the linear-deformation algorithm module. With the algorithm modules, the problem is set up and handed off to an algorithm (in this case linear static solution).  Then for postprocessing another set of algorithms can be invoked.

In [241]:
using FinEtools
using FinEtools.AlgoDeforLinearModule

A few  input parameters are defined: the material parameters.

In [242]:
E = 1.0;
nu = 1.0/3;

The geometry of the tapered panel.

In [243]:
width = 48.0; height = 44.0; thickness  = 1.0;
free_height  = 16.0;
Mid_edge  = [48.0, 52.0];# Location of tracked  deflection

The tapered panel is loaded along the free edge with a unit force, which is here converted to loading per unit area.

In [244]:
magn = 1.0/free_height/thickness;# Magnitude of applied load

For the above input parameters the converged displacement of the tip  of the tapered panel in the direction of the applied shear load is

In [245]:
convutip = 23.97;

The mesh is generated as a rectangular block to begin with, and then the coordinates of the nodes are tweaked into the tapered panel shape. In this case we are using quadratic triangles.

In [246]:
n = 10; # number of elements per side
fens,fes = T6block(width, height, n, n)

# Reshape into a trapezoidal panel
for i = 1:count(fens)
    fens.xyz[i,2] = fens.xyz[i,2]+(fens.xyz[i,1]/width)*(height -fens.xyz[i,2]/height*(height-free_height));
end

The  boundary conditions  are applied to selected finite element nodes.   The selection is based on the inclusion in a selection box.   The  selected nodes are then used twice,  to fix the degree of freedom  in the direction 1 and  in the direction 2.

In [247]:
tolerance = minimum([width, height])/n/1000.;#Geometrical tolerance
# Clamped edge of the membrane
l1 = selectnode(fens; box=[0.,0.,-Inf, Inf], inflate = tolerance);

The list of nodes  is then used to construct entries  for the essential boundary conditions.  The  data is stored in  dictionaries: `ess1` and `ess2 `.  These dictionaries  are used below to compose the computational model.

In [248]:
ess1 = FDataDict("displacement"=>  0.0, "component"=> 1, "node_list"=>l1);
ess2 = FDataDict("displacement"=>  0.0, "component"=> 2, "node_list"=>l1);

The traction boundary condition is applied to the finite elements on the boundary of the panel. First we generate the three-node curve elements on the boundary.

In [249]:
# Traction on the opposite edge
boundaryfes =  meshboundary(fes);

Then from these finite elements we choose the ones that are inside the box that captures the edge of the geometry to which the traction should be applied..

In [250]:
Toplist  = selectelem(fens, boundaryfes, box= [width, width, -Inf, Inf ], inflate=  tolerance);

To apply the traction we create a finite element model machine (FEMM). For the evaluation of the traction it is sufficient to create a the base FEMM.  It consists of the geometry data `GeoD` (connectivity,  integration rule, evaluation  of the basis functions  and basis function gradients with respect to the parametric coordinates), which in turn is composed of the list of the finite elements and  an appropriate quadrature rule (Gauss rule here).

In [251]:
el1femm = FEMMBase(GeoD(subset(boundaryfes, Toplist), GaussRule(1, 3), thickness));

The traction boundary condition is specified with a constant traction vector and the FEMM that will be used to evaluate  the load vector.

In [252]:
flux1 = FDataDict("traction_vector"=>[0.0,+magn],
    "femm"=>el1femm
    );

We make the dictionary for the region (the interior of the domain).  The FEMM and the material are needed. The geometry data  now is equipped with the  triangular  three-point rule. Note the model-reduction type which is used to dispatch to appropriate specializations of the material routines and the FEMM which needs to execute different code for different reduced-dimension models.

In [253]:
# Make the region
MR = DeforModelRed2DStress
material = MatDeforElastIso(MR,  0.0, E, nu, 0.0)
region1 = FDataDict("femm"=>FEMMDeforLinear(MR,
    GeoD(fes, TriRule(3), thickness), material));

The model data is a dictionary.   In the present example it consists of the node set, the array of dictionaries for the regions, and arrays of dictionaries for each essential and natural boundary condition.

In [254]:
modeldata = FDataDict("fens"=>fens,
 "regions"=>[region1],
 "essential_bcs"=>[ess1, ess2],
 "traction_bcs"=>[flux1]
 );

When the model data is defined, we simply pass it to the algorithm.

In [255]:
# Call the solver
modeldata = AlgoDeforLinearModule.linearstatics(modeldata);

The model data is augmented in the algorithm by the nodal field representing the geometry and the displacement field  computed by solving the system of linear algebraic equations of equilibrium.

In [256]:
u = modeldata["u"];
geom = modeldata["geom"];

The complete information returned from the algorithm  is

In [257]:
modeldata

Dict{String,Any} with 7 entries:
  "work"          => 11.9988
  "geom"          => FinEtools.NodalFieldModule.NodalField{Float64}([0.0 0.0; 4…
  "fens"          => FinEtools.FENodeSetModule.FENodeSet([0.0 0.0; 4.8 4.4; … ;…
  "traction_bcs"  => Dict{String,Any}[Dict{String,Any}(Pair{String,Any}("tracti…
  "u"             => FinEtools.NodalFieldModule.NodalField{Float64}([0.0 0.0; 0…
  "essential_bcs" => Dict{String,Any}[Dict{String,Any}(Pair{String,Any}("compon…
  "regions"       => Dict{String,Any}[Dict{String,Any}(Pair{String,Any}("femm",…

Now we can extract the displacement at the mid-edge node and compare to the converged (reference) value..

In [258]:
# Extract the solution
nl = selectnode(fens, box=[Mid_edge[1],Mid_edge[1],Mid_edge[2],Mid_edge[2]],
          inflate=tolerance);
theutip = u.values[nl,:]
println("displacement =$(theutip[2]) as compared to converged $convutip")

displacement =23.93976266124016 as compared to converged 23.97


For postprocessing  we will export a VTK file  with the displacement field (vectors)  and  one scalar field ($\sigma_{xy}$).

In [259]:
modeldata["postprocessing"] = FDataDict("file"=>"cookstress",
   "quantity"=>:Cauchy, "component"=>:xy);
modeldata = AlgoDeforLinearModule.exportstress(modeldata);

The  attribute `"postprocessing"` holds additional data computed and returned by the algorithm:

In [260]:
modeldata["postprocessing"]

Dict{String,Any} with 5 entries:
  "file"            => "cookstress"
  "component"       => :xy
  "exported_files"  => String["cookstress-Cauchyxy-region 1.vtk"]
  "exported_fields" => FinEtools.FieldModule.Field[FinEtools.NodalFieldModule.N…
  "quantity"        => :Cauchy

Provided we have  `paraview` in the PATH, we can bring it up  to display the exported data.

In [261]:
File = modeldata["postprocessing"]["exported_files"][1]
@async run(`"paraview.exe" $File`);

We can also extract the minimum and maximum value of the shear stress.

In [262]:
fld = modeldata["postprocessing"]["exported_fields"][1]
println("$(minimum(fld.values)) $(maximum(fld.values))")

-0.06292574794975273 0.12022571940768388
